# Integration with LangChain

This notebook demonstrates how to use the WRITER ecosystem within LangChain, including setup, chat model usage, and tool invocation.

## Prerequisites

Before getting started, ensure the following prerequisites are met:

- A [Writer AI Studio](https://app.writer.com/register) account
- An API key, which you can obtain by following the [API Quickstart](https://dev.writer.com/api-guides/quickstart)


## 📦 Installation
Install the Writer integration package:

In [ ]:
%pip install langchain-writer langchain langchain_core

## 🔑 Setup API Key

Next, set the `WRITER_API_KEY` environment variable. Setting it in a `.env` file in the root of the project is recommended; however, this tutorial sets it directly as an environment variable if a `.env` file is not present.

In [ ]:
import os
import getpass

if not os.getenv("WRITER_API_KEY"):
    os.environ["WRITER_API_KEY"] = getpass.getpass("Enter your Writer API Key: ")


## Initialize a Writer chat model with LangChain

In [ ]:
from langchain_writer import ChatWriter
from langchain_core.messages import SystemMessage, HumanMessage

model = ChatWriter(model="palmyra-x5")

## 🧠 Chat model usage

The package is now installed and the WRITER API key is configured.
LangChain can now be used to create a chat with a Palmyra model.

The `model` argument in `ChatWriter` specifies which Palmyra model to use, such as `palmyra-x5`, `palmyra-general`, or domain-specific models. You can send structured messages using `SystemMessage` and `HumanMessage`.

> **Best practice:** Use a `SystemMessage` to define the assistant’s role, tone, and output structure. This helps generate more consistent and reliable results, especially when expecting structured JSON outputs.


In [ ]:
sections_prompt = "Generate a blog outline for AI in healthcare."

response = model.invoke([
    SystemMessage(content="You are a blog planning assistant. Generate a structured blog plan based on the user's requirements. Respond with a JSON object containing a 'sections' array where each section has 'name', 'description', and 'main_body' fields."),
    HumanMessage(content=sections_prompt)
])

print(response.content)


## 🚀 LangGraph integration with ChatWriter

This section demonstrates how to build a LangGraph agent using the `ChatWriter` model. The process includes defining tools, the agent state, and constructing the execution graph in a cookbook-friendly style.

### 📦 Step 1: Imports

Import LangChain and LangGraph essentials, including message types, tool decorators, and the graph framework. `Annotated` is used to manage state updates.

In [ ]:
from langchain_writer import ChatWriter
from langchain.tools import tool
from langchain.messages import SystemMessage, HumanMessage, ToolMessage, AnyMessage
from typing_extensions import TypedDict, Annotated
import operator
from langgraph.graph import StateGraph, START, END

### 🛠 Step 2: Define Tools

This step defines a few basic arithmetic tools. In practice, any callable can be defined as a tool. Using the `@tool` decorator allows LangGraph to recognize these as callable nodes in the execution graph.

> Tip: Keep tool interfaces simple and clearly typed to ensure the agent can call them correctly.

In [ ]:
@tool
def add(a: int, b: int) -> int:
    """Return the sum of two integers."""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Return the product of two integers."""
    return a * b

@tool
def divide(a: int, b: int) -> float:
    """Return the result of dividing the first integer by the second."""
    return a / b


tools = [add, multiply, divide]
tools_by_name = {tool.name: tool for tool in tools}
model = ChatWriter(model="palmyra-x5")
model_with_tools = model.bind_tools(tools)


### 📊 Step 3: Define Agent State

LangGraph uses a structured `state` object to keep track of messages and other context. Using `Annotated` with `operator.add` ensures that new messages are appended rather than overwriting existing ones.

> Tip: You can extend the state with custom fields if your agent needs additional context.

In [ ]:
class MessagesState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]
    llm_calls: int

### 💬 Step 4: Define LLM Node

This node represents the LLM call. It decides if tools should be invoked based on the conversation context. The returned dictionary updates the agent state.

> Tip: Keep system instructions clear to guide the LLM on how to handle tool calls and responses.

In [ ]:
def llm_call(state: dict):
    return {
        "messages": [
            model_with_tools.invoke([
                SystemMessage(content="You are a helpful assistant tasked with performing arithmetic on a set of inputs.")
            ] + state["messages"])
        ],
        "llm_calls": state.get("llm_calls", 0) + 1
    }

### 🛠 Step 5: Define Tool Node

The tool node executes any tool calls made by the LLM. Each tool call produces a `ToolMessage` which is appended to the agent state.

In [ ]:
def tool_node(state: dict):
    result = []
    for tool_call in state["messages"][-1].tool_calls:
        tool = tools_by_name[tool_call["name"]]
        observation = tool.invoke(tool_call["args"])
        result.append(ToolMessage(content=observation, tool_call_id=tool_call["id"]))
    return {"messages": result}

### 🔁 Step 6: Define Conditional Edge

This function controls the flow of the graph. If the LLM has requested a tool call, the agent proceeds to the tool node; otherwise, it ends.

In [ ]:
from typing import Literal
def should_continue(state: MessagesState) -> Literal["tool_node", END]:
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        return "tool_node"
    return END

### 🏗 Step 7: Build and Compile Agent

Assemble the graph, add nodes, define edges, and compile the agent.

> Tip: You can visualize this graph before compilation to debug the flow of nodes and edges.

In [ ]:
agent_builder = StateGraph(MessagesState)
agent_builder.add_node("llm_call", llm_call)
agent_builder.add_node("tool_node", tool_node)
agent_builder.add_edge(START, "llm_call")
agent_builder.add_conditional_edges("llm_call", should_continue, ["tool_node", END])
agent_builder.add_edge("tool_node", "llm_call")

agent = agent_builder.compile()

### 🖼 Step 8: Visualize

Use the built-in graph visualization to see the execution flow. This is especially helpful for complex agents with many nodes.

In [ ]:
from IPython.display import Image, display
display(Image(agent.get_graph(xray=True).draw_mermaid_png()))

### Step 9: Test Agent

Send a test message to your agent and print the results. Each message returned can be pretty-printed for readability.

In [ ]:
messages = [HumanMessage(content="Add 3 and 4.")]
messages = agent.invoke({"messages": messages}, {"recursion_limit": 7})
for m in messages["messages"]:
    m.pretty_print()

# messages = agent.invoke({"messages": [{"role": "user", "content": "Add 3 and 4."}]}, {"recursion_limit": 7})
# for m in messages["messages"]:
#     m.pretty_print()

# DeepAgents with ChatWriter and web search

This notebook demonstrates how to build a deep agent using `ChatWriter` and Writer's Web Search tool to perform real-time research and generate reports.

### 📦 Step 1: Imports

Import `ChatWriter`, `create_deep_agent`, and message types.

In [ ]:
%pip install deepagents

In [ ]:
from langchain_writer import ChatWriter
from langchain_core.messages import SystemMessage, HumanMessage
from deepagents import create_deep_agent

### 🛠 Step 2: initialize the model and Writer client

Assumes `WRITER_API_KEY` is already set in the environment.

### 🔍 Step 3: define web search tool

The tool configuration tells the agent how to use the Writer Web Search functionality.

In [ ]:
tools = [{
    "type": "web_search",
    "function": {
        "include_domains": ["www.ibm.com/quantum"],  # Optional: restrict to certain domains
        # "exclude_domains": []   # Optional: exclude domains
    }
}]

### 🤖 Step 4: create deep agent

Use `create_deep_agent` with the ChatWriter model and the Web Search tool.

In [ ]:
system_prompt = '''
You are an expert researcher. Your job is to answer questions accurately and provide references.
You have access to the Web Search tool to retrieve up-to-date information from the internet.
'''

agent = create_deep_agent(
    model=model,
    tools=tools,
    system_prompt=system_prompt
)

### 🏃 Step 5: run the agent with a query

Send a research question to the agent and get a response using Web Search.

In [ ]:
result = agent.invoke({"messages": [{"role": "user", "content": "Find recent breakthroughs in quantum computing. Include references from at most 3 sources."}]})
print(result['messages'][-1].content)